In [2]:
import cdsapi

In [ ]:
c = cdsapi.Client()

dataset = 'reanalysis-era5-single-levels'
request = {
     'product_type': 'reanalysis',
     'variable': [
            '2m_temperature',
            '2m_dewpoint_temperature',
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
            'total_precipitation'
        ],
     'year': '2025',
     'month': '03',
     'day': '01',
     'time': '12:00',
     'area': [24, 102, 8, 110],
     'format': 'netcdf', 
}
target = 'download.zip'

c.retrieve(dataset, request, target)

2025-09-18 15:46:01,866 INFO Request ID is d13158d7-d73a-4e63-a7c9-2bcf0fd82639
2025-09-18 15:46:02,209 INFO status has been updated to accepted
2025-09-18 15:46:11,665 INFO status has been updated to successful


171ca24e6e71c119bff3d7daf9be1a98.zip:   0%|          | 0.00/95.6k [00:00<?, ?B/s]

'download.zip'

In [2]:
import xarray as xr
import pandas as pd

In [25]:
# mở file instant
ds_instant = xr.open_dataset("data_stream-oper_stepType-instant.nc", engine='netcdf4')
print(ds_instant)

<xarray.Dataset>
Dimensions:     (valid_time: 1, latitude: 65, longitude: 33)
Coordinates:
    number      int64 ...
  * valid_time  (valid_time) datetime64[ns] 2025-03-01T12:00:00
  * latitude    (latitude) float64 24.0 23.75 23.5 23.25 ... 8.75 8.5 8.25 8.0
  * longitude   (longitude) float64 102.0 102.2 102.5 ... 109.5 109.8 110.0
    expver      object ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 ...
    d2m         (valid_time, latitude, longitude) float32 ...
    u10         (valid_time, latitude, longitude) float32 ...
    v10         (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-18T08:38 GRIB to CDM+CF via cfgrib-0.9.1...


In [ ]:
# mở file accum
ds_accum = xr.open_dataset("data_stream-oper_stepType-accum.nc", engine='netcdf4')
print(ds_accum)

<xarray.Dataset>
Dimensions:     (valid_time: 1, latitude: 65, longitude: 33)
Coordinates:
    number      int64 ...
  * valid_time  (valid_time) datetime64[ns] 2025-03-01T12:00:00
  * latitude    (latitude) float64 24.0 23.75 23.5 23.25 ... 8.75 8.5 8.25 8.0
  * longitude   (longitude) float64 102.0 102.2 102.5 ... 109.5 109.8 110.0
    expver      object ...
Data variables:
    tp          (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-18T08:38 GRIB to CDM+CF via cfgrib-0.9.1...


In [ ]:
# gộp 2 dataset
ds = xr.merge([ds_instant, ds_accum])
print(ds)

<xarray.Dataset>
Dimensions:     (valid_time: 1, latitude: 65, longitude: 33)
Coordinates:
    number      int64 0
  * valid_time  (valid_time) datetime64[ns] 2025-03-01T12:00:00
  * latitude    (latitude) float64 24.0 23.75 23.5 23.25 ... 8.75 8.5 8.25 8.0
  * longitude   (longitude) float64 102.0 102.2 102.5 ... 109.5 109.8 110.0
    expver      <U4 '0001'
Data variables:
    t2m         (valid_time, latitude, longitude) float32 ...
    d2m         (valid_time, latitude, longitude) float32 ...
    u10         (valid_time, latitude, longitude) float32 ...
    v10         (valid_time, latitude, longitude) float32 ...
    tp          (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-18T

In [31]:
# chuyển sang dataframe
df = ds.to_dataframe().reset_index()

# lưu csv
df.to_csv("era5_merged.csv", index=False)

df.head()

,valid_time,latitude,longitude,number,expver,t2m,d2m,u10,v10,tp
0,2025-03-01 12:00:00,24.0,102.00,0,0001,295.443115,278.860352,1.968323,1.546448,0.0
1,2025-03-01 12:00:00,24.0,102.25,0,0001,293.859131,277.825195,1.513245,1.536682,0.0
2,2025-03-01 12:00:00,24.0,102.50,0,0001,294.734131,275.725586,2.104065,1.878479,0.0
3,2025-03-01 12:00:00,24.0,102.75,0,0001,295.066162,275.834961,2.188049,1.862854,0.0
4,2025-03-01 12:00:00,24.0,103.00,0,0001,295.425537,277.348633,1.754456,1.806213,0.0


In [4]:
df = pd.read_csv('era5_merged.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   valid_time  2145 non-null   object 
 1   latitude    2145 non-null   float64
 2   longitude   2145 non-null   float64
 3   number      2145 non-null   int64  
 4   expver      2145 non-null   int64  
 5   t2m         2145 non-null   float64
 6   d2m         2145 non-null   float64
 7   u10         2145 non-null   float64
 8   v10         2145 non-null   float64
 9   tp          2145 non-null   float64
dtypes: float64(7), int64(2), object(1)
memory usage: 167.7+ KB
